<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-23 15:42:55
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 43.00 K (0.3%)
Current PnL: -32.99 L (-20.62%)
CY Booked + Current PnL: -17.31 L (-10.82%)
-------------------
Total profit:  1.45 L
Total loss:  -34.44 L
-------------------
Total Booked + Current PnL: 9.67 L (7.27%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.09%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.60 L (70.42%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 3.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.62,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,61.0,H-SC,6.14,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
18,COALINDIA,0.58,4.76,12.33,17.67,19435.0,7160.0,157620.0,478.51,26.26,55.0,L-LC,11.42,185.0,0.37,1.13,26.53,XY25,ATH,MINING
77,TTKPRESTIG,-0.85,-31.13,45.34,0.09,31468.0,-31373.0,69404.0,770.00,61.15,29.0,M-SC,10.09,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
36,ICICIGI,0.91,3.50,15.31,19.34,31707.0,6994.0,207103.0,2252.93,-12.97,62.0,X-MC,1.96,66.0,0.22,1.49,19.99,X40,ATH,INSURANCE
51,MEDANTA,3.19,-2.64,27.44,24.08,33915.0,-3354.0,123596.0,1486.00,-3.30,55.0,X-SC,9.77,87.0,-0.10,0.89,13.20,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,4.20,4.52,98.93,107.91,152661.0,6668.0,154312.0,1641.55,-6.14,63.0,M-SC,8.10,220.0,0.04,1.11,16.09,OX40N,ATH,CABLES
76,TRIDENT,4.07,-28.86,80.65,28.51,52929.0,-26626.0,65628.0,48.00,-8.51,50.0,M-SC,8.45,223.0,-0.50,0.47,12.30,XR,NTT,TEXTILES
49,LAOPALA,3.20,-37.08,121.07,39.09,99867.0,-48618.0,82487.0,464.00,80.09,64.0,H-SC,14.13,154.0,-0.49,0.59,10.31,AR,NTT,CERAMICS
51,MEDANTA,3.19,-2.64,27.44,24.08,33915.0,-3354.0,123596.0,1486.00,-3.30,55.0,X-SC,9.77,87.0,-0.10,0.89,13.20,XY24,NTT,HEALTHCARE
65,SIEMENS,2.99,-11.28,41.47,25.51,68471.0,-20985.0,165110.0,4671.50,38.57,62.0,H-LC,1.99,51.0,-0.31,1.19,21.67,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,-4.27,-26.51,78.62,31.26,133327.0,-61184.0,169584.0,18931.72,-67.16,37.0,X-MC,24.78,54.0,-0.46,1.22,4.08,X40N,ATH,IT
72,TATAELXSI,-2.98,-32.06,94.34,32.03,84497.0,-42269.0,89566.0,9161.00,-14.99,31.0,H-SC,8.62,158.0,-0.50,0.64,0.32,OX40N,NTT,IT
30,HAPPSTMNDS,-2.84,-54.27,304.18,84.85,178298.0,-69549.0,58616.0,1480.71,-45.69,31.0,H-SC,26.00,152.0,-0.39,0.42,0.00,AR,ATH,IT
69,SURYODAY,-2.71,-32.34,81.83,23.03,99150.0,-57905.0,121166.0,216.00,31.58,26.0,H-SC,4.52,166.0,-0.58,0.87,20.22,XR,NTT,BANKS
80,VALIANTORG,-2.41,-50.05,248.63,74.15,203195.0,-81879.0,81726.0,838.00,-247.22,41.0,H-SC,23.59,145.0,-0.40,0.59,4.76,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,4.20,4.52,98.93,107.91,152661.0,6668.0,154312.0,1641.55,-6.14,63.0,M-SC,8.10,220.0,0.04,1.11,16.09,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.85,-31.13,45.34,0.09,31468.0,-31373.0,69404.0,770.00,61.15,29.0,M-SC,10.09,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
66,SIS,0.99,-31.16,76.15,21.26,58206.0,-34596.0,76436.0,528.00,1793.78,33.0,H-SC,4.59,168.0,-0.59,0.55,5.38,OX40N,NTT,MISC
72,TATAELXSI,-2.98,-32.06,94.34,32.03,84497.0,-42269.0,89566.0,9161.00,-14.99,31.0,H-SC,8.62,158.0,-0.50,0.64,0.32,OX40N,NTT,IT
40,INDIGOPNTS,0.73,-32.45,48.07,0.02,56680.0,-56647.0,117912.0,1408.00,79.69,28.0,M-SC,10.35,222.0,-1.00,0.85,2.56,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,-0.56,20.33,20.66,45.2,47669.0,38988.0,230730.0,1856.0,37.8,63.0,X-MC,6.68,79.0,0.82,1.66,28.51,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.58,4.76,12.33,17.67,19435.0,7160.0,157620.0,478.51,26.26,55.0,L-LC,11.42,185.0,0.37,1.13,26.53,XY25,ATH,MINING
26,FINCABLES,4.20,4.52,98.93,107.91,152661.0,6668.0,154312.0,1641.55,-6.14,63.0,M-SC,8.10,220.0,0.04,1.11,16.09,OX40N,ATH,CABLES
50,MASFIN,2.62,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,61.0,H-SC,6.14,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
1,ABB,-1.16,8.57,34.08,45.57,96801.0,22421.0,284040.0,7934.00,-30.22,64.0,H-MC,6.74,125.0,0.23,2.04,26.13,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.62,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,61.0,H-SC,6.14,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
1,ABB,-1.16,8.57,34.08,45.57,96801.0,22421.0,284040.0,7934.00,-30.22,64.0,H-MC,6.74,125.0,0.23,2.04,26.13,AR,NTT,ELECTRICAL
26,FINCABLES,4.20,4.52,98.93,107.91,152661.0,6668.0,154312.0,1641.55,-6.14,63.0,M-SC,8.10,220.0,0.04,1.11,16.09,OX40N,ATH,CABLES
39,INDIAMART,1.45,-4.72,116.96,106.72,137448.0,-5819.0,117517.0,4810.62,-57.03,51.0,H-SC,7.94,140.0,-0.04,0.85,17.56,AR,ATH,MISC
85,ZYDUSLIFE,0.80,-4.25,41.66,35.63,83594.0,-8917.0,200657.0,1286.17,-16.67,52.0,H-MC,5.16,120.0,-0.11,1.44,11.71,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.90,-9.63,48.55,34.24,137279.0,-30130.0,282758.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.03,0.00,X40,NTT,IT
8,AWL,-1.44,-36.59,147.88,57.19,305835.0,-119314.0,206813.0,485.00,-63.67,23.0,X-SC,17.91,81.0,-0.39,1.49,0.00,XY24,NTT,FMCG
33,HCLTECH,-0.72,-7.45,33.80,23.83,75683.0,-18011.0,223913.0,1908.19,-2.81,24.0,X-LC,3.85,8.0,-0.24,1.61,7.43,X40,ATH,IT
73,TCS,-0.37,-23.29,61.10,23.58,202767.0,-100768.0,331861.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.39,0.00,X40,ATH,IT
9,BAJAJHFL,-0.92,-25.25,108.00,55.47,196398.0,-61436.0,181850.0,181.50,-35.82,31.0,X-MC,17.65,60.0,-0.31,1.31,0.00,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,2.73,4.08,16.03,20.77,39066.0,9563.0,243707.0,1644.00,-7.70,58.0,X-MC,0.61,64.0,0.24,1.75,11.79,X40N,NTT,BREWERIES
21,DABUR,0.91,0.56,42.90,43.70,107018.0,1392.0,249460.0,735.00,-8.76,52.0,X-MC,1.34,70.0,0.01,1.79,15.70,XY24,BTT,FMCG
36,ICICIGI,0.91,3.50,15.31,19.34,31707.0,6994.0,207103.0,2252.93,-12.97,62.0,X-MC,1.96,66.0,0.22,1.49,19.99,X40,ATH,INSURANCE
65,SIEMENS,2.99,-11.28,41.47,25.51,68471.0,-20985.0,165110.0,4671.50,38.57,62.0,H-LC,1.99,51.0,-0.31,1.19,21.67,AR,ATH,ELECTRICAL
75,TMPV,0.52,-15.57,57.92,33.33,159329.0,-50716.0,275084.0,600.00,-82.53,59.0,X-LC,2.68,4.0,-0.32,1.98,12.46,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,-0.92,-25.25,108.00,55.47,196398.0,-61436.0,181850.0,181.50,-35.82,31.0,X-MC,17.65,60.0,-0.31,1.31,0.0,X40N,ATH,FINANCE
8,AWL,-1.44,-36.59,147.88,57.19,305835.0,-119314.0,206813.0,485.00,-63.67,23.0,X-SC,17.91,81.0,-0.39,1.49,0.0,XY24,NTT,FMCG
53,PGHH,-0.06,-13.87,53.27,32.01,97966.0,-29616.0,183904.0,17616.87,-41.76,32.0,X-MC,9.55,57.0,-0.30,1.32,0.0,X40,ATH,FMCG
42,INFY,-1.90,-9.63,48.55,34.24,137279.0,-30130.0,282758.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.03,0.0,X40,NTT,IT
73,TCS,-0.37,-23.29,61.10,23.58,202767.0,-100768.0,331861.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.39,0.0,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.64,-31.41,108.26,42.84,48269.0,-20420.0,44586.0,424.00,-54.82,47.0,X-SC,17.86,80.0,-0.42,0.32,7.25,XY24,NTT,MISC
58,RELAXO,-0.32,-53.79,229.60,52.31,154011.0,-78082.0,67078.0,1176.00,-50.83,39.0,X-SC,14.06,92.0,-0.51,0.48,0.56,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.06,-47.60,158.02,35.20,106545.0,-61245.0,67425.0,2096.00,-11.62,32.0,X-SC,21.81,91.0,-0.57,0.49,0.06,X40,NTT,FOOTWEAR
51,MEDANTA,3.19,-2.64,27.44,24.08,33915.0,-3354.0,123596.0,1486.00,-3.30,55.0,X-SC,9.77,87.0,-0.10,0.89,13.20,XY24,NTT,HEALTHCARE
35,HONAUT,2.18,-22.47,84.85,43.32,107149.0,-36592.0,126280.0,58357.33,-31.88,44.0,X-SC,11.53,90.0,-0.34,0.91,2.53,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.49,-14.29,38.91,19.06,113445.0,-48608.0,291558.0,452.00,-53.09,49.0,X-LC,17.91,1.0,-0.43,2.10,5.57,X40,NTT,FMCG
73,TCS,-0.37,-23.29,61.10,23.58,202767.0,-100768.0,331861.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.39,0.00,X40,ATH,IT
42,INFY,-1.90,-9.63,48.55,34.24,137279.0,-30130.0,282758.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.03,0.00,X40,NTT,IT
75,TMPV,0.52,-15.57,57.92,33.33,159329.0,-50716.0,275084.0,600.00,-82.53,59.0,X-LC,2.68,4.0,-0.32,1.98,12.46,XY24,NTT,AUTO
33,HCLTECH,-0.72,-7.45,33.80,23.83,75683.0,-18011.0,223913.0,1908.19,-2.81,24.0,X-LC,3.85,8.0,-0.24,1.61,7.43,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.68,-32.10,95.71,32.89,50177.0,-24780.0,52426.0,1800.00,-346.50,52.0,L-MC,11.29,258.0,-0.49,0.38,44.38,XR,NTT,BANKS
67,SONACOMS,0.74,-7.27,49.99,39.09,46895.0,-7350.0,93809.0,804.02,-26.55,65.0,M-MC,9.12,193.0,-0.16,0.67,32.71,AR,ATH,AUTO
50,MASFIN,2.62,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,61.0,H-SC,6.14,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
18,COALINDIA,0.58,4.76,12.33,17.67,19435.0,7160.0,157620.0,478.51,26.26,55.0,L-LC,11.42,185.0,0.37,1.13,26.53,XY25,ATH,MINING
37,ICICIPRULI,1.97,9.90,18.95,30.73,41029.0,19508.0,216513.0,790.00,-19.66,60.0,X-MC,4.17,78.0,0.48,1.56,23.93,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-1.72,-49.56,188.84,45.70,168423.0,-87627.0,89188.0,26.40,-7.37,71.0,M-SC,19.07,204.0,-0.52,0.64,46.95,XY24,NTT,TRAVEL
1,ABB,-1.16,8.57,34.08,45.57,96801.0,22421.0,284040.0,7934.00,-30.22,64.0,H-MC,6.74,125.0,0.23,2.04,26.13,AR,NTT,ELECTRICAL
7,ATULAUTO,1.06,-17.15,75.71,45.57,123649.0,-33813.0,163319.0,844.00,3537.84,54.0,M-SC,3.96,248.0,-0.27,1.17,23.59,XY24,NTT,AUTO
67,SONACOMS,0.74,-7.27,49.99,39.09,46895.0,-7350.0,93809.0,804.02,-26.55,65.0,M-MC,9.12,193.0,-0.16,0.67,32.71,AR,ATH,AUTO
82,VOLTAS,-0.56,20.33,20.66,45.20,47669.0,38988.0,230730.0,1856.00,37.80,63.0,X-MC,6.68,79.0,0.82,1.66,28.51,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.67
1,25,45.44
2,50,77.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.13
MC    30.08
LC    24.78
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.99
X40      24.02
X40N     14.78
AR        9.00
XY25      8.78
XR        8.47
OX40N     7.11
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.66
H-SC    22.13
X-LC    21.43
X-SC    11.54
M-SC    10.72
H-MC     4.98
M-MC     2.06
H-LC     1.19
L-LC     1.13
M-LC     1.03
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.59,-13.87,52.38
FINANCE,13.09,-14.84,60.03
IT,11.42,-37.98,107.13
MISC,6.91,-35.44,90.85
ELECTRICAL,6.27,-7.62,46.89
PAINTS,5.03,-31.97,51.58
INSURANCE,4.85,-0.69,35.15
PHARMA,4.10,-3.01,35.43
AUTO,3.82,-17.26,61.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3373734.0,21
X40,1383078.0,15
AR,1320232.0,10
XR,1314126.0,12
X40N,1066499.0,10
OX40N,803123.0,10
XY25,397009.0,6
SR,302739.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3716488.0,24
M-SC,1427938.0,14
X-MC,1362347.0,15
X-LC,1295208.0,12
X-SC,1199374.0,10
H-MC,385625.0,3
L-SC,188194.0,2
M-LC,148803.0,1
M-MC,98480.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1261092.0      6
           AR         949928.0      5
           XR         845620.0      7
M-SC       XY24       826049.0      6
X-LC       X40        670003.0      6
X-SC       XY24       616739.0      4
X-MC       X40        535856.0      7
           X40N       431564.0      4
X-SC       X40N       405416.0      4
H-SC       OX40N      357109.0      4
M-SC       OX40N      350669.0      5
H-SC       SR         302739.0      2
X-LC       XY24       270449.0      2
           X40N       229519.0      2
H-MC       XY24       205230.0      1
X-MC       XY25       200752.0      2
           XY24       194175.0      2
H-MC       AR         180395.0      2
X-SC       X40        177219.0      2
M-SC       XR         176677.0      2
M-LC       XR         148803.0      1
X-LC       XY25       125237.0      2
L-SC       OX40N       95345.0      1
           XR          92849.0      1
M-SC       AR          74543.0      1
H-LC       AR          68471.0      1
M-MC       XY25        51585.0      1
L-MC       XR          50177.0      1
M-MC       AR          46895.0      1
L-LC       XY25        19435.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
